In [4]:
from __future__ import print_function
from pprint import pprint
import os
import csv
import cPickle as pickle

import numpy as np
import scipy as sp

from feature_label_builder import FeatureLabelBuilder, get_feature_keys
from feature_label_builder import data_path, full_path
from util_eval import multiclass_log_loss, multiclass_accuracy

import xgboost as xgb

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load data
(all_xs, all_ys) = pickle.load(open(full_path("train_37_xs_ys_np_sorted.p"), "rb"))
all_ys = all_ys - 1.0

# set up split portion of train and test data
test_percentage = 0.01
all_num = len(all_ys)
train_num = int(round((1. - test_percentage) * all_num))
test_num = all_num - train_num

# prepare train and test dataset
train_xs = all_xs[:train_num]
train_ys = all_ys[:train_num]
test_xs = all_xs[train_num:]
test_ys = all_ys[train_num:]

# convert to xgb matrix
dtrain = xgb.DMatrix(train_xs, label=train_ys)
dtest = xgb.DMatrix(test_xs, label=test_ys)

In [3]:
# params = {'bst:max_depth': 5,
#           'bst:eta': 0.1,
#           'silent': 0,
#           'objective': 'multi:softprob',
#           'num_class': 5,
#           'nthread': 16, # change this!
#           'eval_metric': 'mlogloss',
#           'subsample': 0.01}

classifier = xgb.XGBClassifier(max_depth=3,
                               learning_rate=0.1,
                               n_estimators=2,
                               silent=False,
                               objective='multi:softprob',
                               nthread=4,
                               subsample=0.0001)
classifier.fit(train_xs, train_ys,
               eval_set=[(train_xs, train_ys), (test_xs, test_ys)], 
               eval_metric='mlogloss', 
               verbose=True)

[0]	validation_0-mlogloss:1.381235	validation_1-mlogloss:1.385584
[1]	validation_0-mlogloss:1.202544	validation_1-mlogloss:1.217052


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=2, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=0.0001)